# Chatbot-example

Import

In [1]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import sqlite3

First we extract the user input and the replies from the corpus, and put them into a database.

In [8]:
con = sqlite3.connect('bigbang.db')
con.text_factory = str
cur = con.cursor()
# cur.execute("""
# DROP TABLE IF EXISTS corpus
# ;
# """)
cur.execute("""
CREATE TABLE IF NOT EXISTS 
corpus(input, output)
;
""")
               
with open('big_bang_test.txt') as f:
    lines = f.readlines()

    for line in lines:
        # print(line)
        if "User" in line:
            input = line[(line.index(":")+2):-2]
            output = []
            continue
        if line == "\r\n":
            output = [one.replace("Esmerelda Quest Bot Says: ", "").replace("\r\n", "") for one in output]
            output = ''.join(map(str, output))
            cur.execute("INSERT INTO corpus VALUES (?, ?)", (input, output))
            input = None
            # break
        if input:
            output.append(line)

con.commit()
con.close()

We extract sentences from the dataset.

In [10]:
con = sqlite3.connect('bigbang.db')
cur = con.cursor()
result = []
response = []

for row in con.execute("SELECT input FROM corpus"):
    # print(row)
    result.append(row)

for row in con.execute("SELECT output FROM corpus"):
    # print(row)
    response.append(row)
con.close()


In [11]:
result = [one[0] for one in result]
response = [one[0] for one in response]

In [14]:
result

['Can you give me another hint about favorite episodes',
 'I like it when Raj and Lucy text date',
 'Prom websites',
 'sorry, autocorrect',
 'Can you give me a hint about when Raj and Lucy text date',
 'Can you give me a hint about when Bert talks to Penny',
 'I like it when Bert talks to Penny',
 'If you steal my idol',
 '6 tons of granite and micacious schist',
 'I will roll right over you',
 'I like Thor and Doctor Jones',
 'One plays with lightning',
 'The Justice League one',
 'A costume contest',
 'I’m Superman',
 'Sheldon is the Flash',
 'I’m pacing',
 'Raj is Aquaman',
 'Aquaman sucks under water',
 'Leonard is Green Lantern',
 'Glasses',
 'Howard is Batman',
 'Penny is Wonder Woman',
 'I like it when Leonard talks to Penny about Wonder Woman']

This is what we would do if we simply read from the txt.

In [ ]:
# i = 0
# xinyi = []

# with open("big_bang_test.txt") as f:
#     conversation = f.readlines()
#     for sent in conversation:
#         if re.match("Xinyi the User says:",sent) is not None:
#             xinyi.append(sent[21:-1])
# print(xinyi)

Main function:

In [13]:
def compare_similarity(corpus, reply, text):
    """
    This function compares the similarity between the given inquiry with the conversation corpus.

    ----------------------------------
    corpus: list
        a list of inquiries.
    text: string
        user input
    """

    new_corpus = corpus.copy()
    new_corpus.append(text)
    tfidf = TfidfVectorizer().fit_transform(new_corpus)
    pairwise_similarity = tfidf * tfidf.T 
    arr = pairwise_similarity.toarray() 
    np.fill_diagonal(arr, np.nan)

    result_idx = np.nanargmax(arr[-1])

    return (new_corpus[result_idx], reply[result_idx])


Example: when the user says "raj lucy dating", the chat bot would assume the default topic 'I like it when Raj and Lucy text date', and replies accordingly.

In [16]:
input = "raj lucy dating"
compare_similarity(result, response, input)

('I like it when Raj and Lucy text date',
 'At the library, so her shyness won’t be as obvious, and since he can’t speak to women, he can text back and forth with her. Lucy asks him, I don’t know, do you visit a lot of porn websites?')

Chatbot function and some simple examples

In [38]:
def chatbot():
    from builtins import input
    user = input("User: ")
    print("Chatbot: "+compare_similarity(result, response, user)[1])

In [39]:
chatbot()

User: raj lucy date
Chatbot: At the library, so her shyness won’t be as obvious, and since he can’t speak to women, he can text back and forth with her. Lucy asks him, I don’t know, do you visit a lot of porn websites?


In [40]:
chatbot()

User: superman
Chatbot: With lifts in my boots, I’ll be taller this year. So they start showing up in costume, and they have really good costumes. Who is the Flash?


In [41]:
chatbot()

User: Prom
Chatbot: Sorry, autocorrect. And then Raj gushes over actually visiting prom websites, and HE says


In [42]:
chatbot()

User: Justice League
Chatbot: They were in the comic book store and Stuart announces there is going to be a contest
